# Section 5 - Canny FRCNN

In this section, we utilize the OpenCV implementation of the Canny algorithm as a preprocessing layer to Faster-RCNN and analyze its performance.  Canny is a popular multi-stage edge detection algorithm designed to a) highlight the edges of all objects within an image and b) remove anything between different edges. 

In using this algorithm, we hypothesize that FRCNN will achieve better performance on more complex images since much of the noise of non-targets is removed and only the outline of the edges is considered.  By training on a ship's outline, which is relatively consistent regardless of ship sizes, we expect the model to identify targets in areas that previously had significant amounts of noise, such as docks, inclement weather, and other confusers.

## 5.1 - Canny Overview

The Canny pipeline includes three steps:

1. **Noise Reduction**: Use Gaussian Blurring on the entire image to reduce noise surrounding edges that may lead to false positives.
2. **Gradient Intensity**: It then uses the Sobel kernel (another Edge Detection method) in the X and Y directions to determine the gradient magnitude. Then it uses a formula to extract the gradient direction of individual pixels.
3. **Non-Maximum Suppression**: Once edges are proposed, NMS is used to remove any pixels that may be a false positive edge.

For more details on the Canny algorithm, OpenCV's documentation has a good explanation of the topic: https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html

In [ ]:
# Utility Imports
import os, sys
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import cv2

project_path = pathlib.Path.cwd().parent.resolve().parent.resolve()
print(f"Project path: {project_path}")

sys.path.append(os.path.join(project_path, "src", "frcnn"))
sys.path.append(os.path.join(project_path, "src", "utils"))

# Data Handling Imports
from SeaSarFRCNN import SeaSarFRCNN
from ModelWorker import ModelWorkerFRCNN
from WorkspaceManager import WorkspaceManager

# Model and Model Utility Imports
import torchvision
import torch
from frcnn_models import CannyFRCNN



In [ ]:
workspace_builder = WorkspaceManager()
workspace_builder.run_setup()

if not os.path.exists(workspace_builder._data_path):
    raise FileNotFoundError(f"Not able to find data directory at path: {workspace_builder._data_path}")
else:
    print(f"Using data path: {workspace_builder._data_path}")

## 5.2 - Data Analysis

Below, we provide an example of how Canny can assist the detection models by remove noise and highlight ship outlines.  In the original image, targets are intermingled with what appears to be either inclement weather or some other noise type.  In the succeeding subplot, we can see those inclement weather spots are fully removed while the outline of the targets are retained.

In [ ]:
# Validation Path retrieval
tester_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_annotations = os.path.join(workspace_builder._data_path, "annotations", "train_annotations.coco.json")
train_images = os.path.join(workspace_builder._data_path, "train")

train_data = SeaSarFRCNN(train_images, train_annotations, tester_transforms)

In [ ]:
image, target = train_data._get_image(3270, True, True)

In [ ]:
gaussian_image = cv2.GaussianBlur(np.array(image), ksize=(5, 5), sigmaX=2.0)
canny_image = cv2.Canny(gaussian_image, 200, 100)
train_data.show_edited_subplot([image, gaussian_image, canny_image], ["Original", "Gaussian Blurred", "Post-Canny"], "", "")

### 5.2.1 - False Positives

As shown in 5.3, setting the Canny thresholds to (200, 100) began yield positive results like the image above.  However, there are some drawbacks to using static threshold values. Setting the threshold too high can remove key target details or even entire targets. Using a value too small will not remove enough noise and can reduce model performance.  As we see in the below figures, (200, 100) is not a high enough threshold to clear much of the noise from the shoreline, inevitably leading to misclassifications.

In [ ]:
image, target = train_data._get_image(2732, get_image=True)

In [ ]:
gaussian_image = cv2.GaussianBlur(np.array(image), ksize=(5, 5), sigmaX=2.0)
canny_image = cv2.Canny(gaussian_image, 200, 100)
train_data.show_edited_subplot([image, gaussian_image, canny_image], ["Original", "Gaussian Blurred", "Post-Canny"], "", "")

## Section 5.3 Model Construction and Training

In this section, we build the CannyFRCNN model.  We use a 3x3 kernel and standard deviation of 1 for Gaussian Blurring. For Canny, using 200 or higher for threshold 1 and 100 or higher for threshold 2 began to show good results on most images as this is when noise began being filtered out. 

In [ ]:
# Set up train dataloader
train_data = torchvision.datasets.wrap_dataset_for_transforms_v2(train_data, target_keys=("boxes", "labels"))
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True, collate_fn=train_data._packager)

In [ ]:
# Build the baseline model
frcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
input_features = frcnn.roi_heads.box_predictor.cls_score.in_features
frcnn.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(input_features, 2)

# Build the loss function and optimizer
optimizer = torch.optim.Adam(frcnn.parameters(), lr=0.001)

In [ ]:
canny_model = CannyFRCNN(frcnn, 'cuda', (3, 3), 1.0, 200, 100)
worker = ModelWorkerFRCNN(optimizer, canny_model, False)

In [ ]:
worker.train(train_dataloader, num_epochs=1, indices_to_skip=[3736])